# Million Song Dataset

Dataset comes from: http://millionsongdataset.com/. Currently, looking at just the subset. It contains the musical features (not the actual music files). Additionally, it contains an artist musicbrain id.

In [1]:
# million song dataset manipulation
import h5py
import numpy as np
import h5json
import sqlite3
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
import psycopg2

# cover art
import musicbrainzngs
from IPython.display import Image
from IPython.core.display import HTML 
from Levenshtein import distance, ratio

from datetime import datetime
import urllib

# set a real one at some point
musicbrainzngs.set_useragent("thesis", "0.1", "gnaswerdna@gmail.com")

path = '/Users/andrewsang/datasci/thesis'

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100

%matplotlib inline

## Cover Art and Genre

### Cover Art

In [2]:
artist_mbid = '286ec4c2-b5ca-4f85-b331-280a6d73dd14'
msd_release_str = 'Joao Voz E Violato'
track_id = 'TRACCJA128F149A144'

In [3]:
def cover_art_img_url(release_id):
    try:
        return musicbrainzngs.get_image_list(release_id)['images'][0]['image']
    except:
        return np.NaN

## Method 2: Use Data Dump

Alternatively, download dump and recreate using these directions: https://github.com/lalinsky/mbdata. Might need this for postgres: https://stackoverflow.com/a/46971838

In [4]:
import psycopg2

In [5]:
conn = psycopg2.connect(dbname="musicbrainz", user="andrewsang", host="127.0.0.1", port=5432)
print("Connected!")

Connected!


In [6]:
query = """SELECT DISTINCT id as artist_id, gid as artist_mbid, name as artist_name, comment 
           FROM musicbrainz.artist"""
artist = pd.read_sql(query, conn)

query = """SELECT id as release_id, gid as release_mbid, name as release_name, artist_credit as artist_id  
           FROM musicbrainz.release
           GROUP BY 1,2,3,4"""
release = pd.read_sql(query, conn)

query = """SELECT id as cover_art_id, release as release_id
           FROM cover_art_archive.cover_art
           GROUP BY 1,2"""
cover_art = pd.read_sql(query, conn)

In [7]:
release_w_art = (release.merge(cover_art, how='inner', on=['release_id'])
                        .loc[:,['release_id','release_name','artist_id','release_mbid']]
                        .drop_duplicates()
                        .merge(artist, how='inner', on=['artist_id'])
                        .loc[:,['artist_mbid','release_id','release_name','release_mbid']])

In [26]:
# ltma: lyrics_track_metadata_w_art
ltma = lyrics_track_metadata.merge(release_w_art, on=['artist_mbid'])

ltma.loc[:,'lev_dist'] = [distance(k,v) for k,v in zip(ltma.release_name, ltma.release)]
ltma.loc[:,'str_ratio'] = [ratio(k,v) for k,v in zip(ltma.release_name, ltma.release)]
ltma.loc[pd.isnull(ltma.loc[:,'str_ratio']),'str_ratio'] = np.NaN

ltma.loc[:,'search_rnk'] = ltma.groupby(['track_id'])['str_ratio'].rank(ascending=False, method='first')

ltma = ltma.loc[(ltma['str_ratio'] > .5) & (ltma['search_rnk'] == 1), :]

In [31]:
def worker(num):
    print('Started', num, 'at ',datetime.now().strftime('%H:%M:%S'))
    df = split_dataset_into_fourths(num, art_url_df)
    df.loc[:,'cover_url'] = df.loc[:,'release_mbid'].map(lambda s: cover_art_img_url(s))
    now = datetime.now().strftime('%Y%M%d_%H%M%S')
    df.to_csv('{0}/output_{1}_{2}.csv'.format(folder, now, num))
    print('Finished with', num, 'at ',datetime.now().strftime('%H:%M:%S'))
    return

In [32]:
# # run multiprocessing
# n = multiprocessing.cpu_count()
# result = pd.DataFrame()
# folder = 'rel_mb'
# jobs = []
# for i in range(n):
#     p = multiprocessing.Process(target=worker, args=(i,))
#     jobs.append(p)
#     p.start()

In [34]:
url_df = pd.DataFrame()
for f in os.listdir('rel_mb'):
    if f[-4:] == '.csv':
        url_df = pd.concat([url_df, pd.read_csv('rel_mb/'+f)])

FileNotFoundError: [Errno 2] No such file or directory: 'rel_mb'

In [ ]:
for r, url in zip(url_df.release_mbid, url_df.cover_url):
    try:
        urllib.request.urlretrieve(url, "../data/covers/{0}.jpg".format(r))
#         urllib.request.urlretrieve(url, "/volumes/TOSHIBA EXT/data/covers/{0}.jpg".format(r))
    except:
        pass